In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)

os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


# from config.config import SQLQuery
# querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/plaid_credit
/Users/shashankgupta/Documents/code/git_project/plaid_credit/Code


In [2]:
from preprocess import Convert,MissingValues,Outlier,FeatureSelection
from feature_transformation import Scaler,Transform,Selection
from model_building import split_test_train, feature_encoding, classification_models
from model_evaluations import model_metrics, feature_importance, probability_bins, cross_validation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

from statsmodels.stats.outliers_influence import variance_inflation_factor

# object initiation 
tf = Transform()
sel = Selection()
ft = FeatureSelection()
cv = Convert()
mv = MissingValues()
ot = Outlier()

# set seed
# seed = 1

In [3]:
corr_arr = 0.5 # person correlation coefficient # change it to 0.5
vif_arr = 5 # vif coefficient
features_arr = 10 # total number of features to be selected from backward feature selection
iv_upper_limit = 0.5 # upper threshold of iv # change it to 0.6
iv_lower_limit = 0.02 # lower threshold of iv

In [4]:
df_raw = pd.read_pickle('/Users/shashankgupta/Documents/code/git_project/plaid_credit/data/final_dataset_2.pkl')
df_raw.shape

(646, 255)

In [5]:
# excluding stddev features
keep_stddev_cols = ['stddev_credit_amt_3M']
stddev_cols = list(df_raw.loc[:,df_raw.columns.str.contains('stddev')].columns)

keep_cols = list(set(list(df_raw.columns)).difference(set(stddev_cols))) + keep_stddev_cols


df_raw = df_raw[keep_cols]
df_raw.shape

(646, 219)

In [6]:
df_raw['txn_flag'].value_counts()

txn_flag
1.0    637
0.0      9
Name: count, dtype: int64

In [7]:
df_raw = df_raw[df_raw['txn_flag']==1]
df_raw.shape

(637, 219)

In [8]:
df_raw = df_raw.drop(['business_id', 'lending_business_id','decision_date','drawn_flag', 'everDPD_15', 'fico_score','txn_flag'], axis=1)
df_raw.shape

(637, 212)

In [9]:
# get constant features
def get_const_features(df):
    const_list = []
    for col in df.columns: 
        if (len(df[col].unique())==1):
            const_list.append(col)
    return(const_list)

# get quasi-constant features
def get_quasi_const_features(df, threshold=0.01):
    qconst_list = []
    for col in df.columns: 
        if (df[col].var() <= threshold):
            qconst_list.append(col)
    return(qconst_list)

# view missing values
def missing_value(df):
    percent_missing = df.isnull().sum() * 100 / len(df)
    missing_val_df = pd.DataFrame({'percent_missing': percent_missing})
    missing_val_df.sort_values(by='percent_missing', ascending=False, inplace=True)
    return missing_val_df

# get boolean columns
def findbool(df):
    bool_arr = []
    for col in df.columns: 
        if (len(df[col].unique())<=2):
            bool_arr.append(col)
    return(bool_arr)

# get datatypes frequency
def get_datatypes_freq(df):
    type_dct = {str(k): list(v) for k, v in df.groupby(df.dtypes, axis=1)}
    type_dct_info = {k: len(v) for k, v in type_dct.items()}
    return type_dct, type_dct_info

## iv_woe

def iv_woe(data, target, bins=10, show_woe=False):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    #Extract Column Names
    cols = data.columns
    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target])]:
        if (data[ivars].dtype.kind in 'bifc') and (len(np.unique(data[ivars]))>10):
            binned_x = pd.qcut(data[ivars], bins,  duplicates='drop')
            d0 = pd.DataFrame({'x': binned_x, 'y': data[target]})
        else:
            d0 = pd.DataFrame({'x': data[ivars], 'y': data[target]})

        
        # Calculate the number of events in each group (bin)
        d = d0.groupby("x", as_index=False).agg({"y": ["count", "sum"]})
        d.columns = ['Cutoff', 'N', 'Events']
        
        # Calculate % of events in each group.
        d['% of Events'] = np.maximum(d['Events'], 0.5) / d['Events'].sum()

        # Calculate the non events in each group.
        d['Non-Events'] = d['N'] - d['Events']
        # Calculate % of non events in each group.
        d['% of Non-Events'] = np.maximum(d['Non-Events'], 0.5) / d['Non-Events'].sum()

        # Calculate WOE by taking natural log of division of % of non-events and % of events
        d['WoE'] = np.log(d['% of Events']/d['% of Non-Events'])
        d['IV'] = d['WoE'] * (d['% of Events'] - d['% of Non-Events'])
        d.insert(loc=0, column='Variable', value=ivars)
        print("Information value of " + ivars + " is " + str(round(d['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d], axis=0)

        #Show WOE Table
        if show_woe == True:
            print(d)
    return newDF, woeDF


# get feature list after iterative VIF elimination
def vif_iter(df, iv, threshold=10):
    vif_data = pd.DataFrame()
    vif_data["feature"] = iv
    vif_data["VIF"] = [variance_inflation_factor(df[iv].values, i) for i in range(len(iv))]
    if len(vif_data[vif_data['VIF'] == np.inf]) > 0:
        feature = vif_data[vif_data['VIF'] == np.inf]['feature'].iloc[0]
        iv.remove(feature)
        vif_iter(df, iv, threshold)
    elif len(vif_data[vif_data['VIF'] > threshold]) > 0:
        feature = vif_data.sort_values(by='VIF', ascending=False)['feature'].iloc[0]
        iv.remove(feature)
        vif_iter(df, iv, threshold)
    vif_data = pd.DataFrame()
    vif_data["feature"] = iv
    vif_data["VIF"] = [variance_inflation_factor(df[iv].values, i) for i in range(len(iv))]
    return iv, vif_data



In [10]:
def main_func(df_raw, corr_arr,vif_arr,features_arr,iv_upper_limit,iv_lower_limit,seed):
    x_train, y_train, x_test, y_test = split_test_train(df_raw, target_column='target', test_size=0.3, random_state=seed)
    print(f'{x_train.shape = }', '|' ,f'{y_train.shape = }', '|' ,f'{x_test.shape = }', '|' ,f'{y_test.shape = }')


    # copy to df
    df = x_train.copy(deep=True)

    const_list = get_const_features(df)
    df = df.drop(columns=const_list)
    
    qconst_list = get_quasi_const_features(df, threshold=0.01)
    df = df.drop(columns=qconst_list)

    #missing values
    df = df.fillna(0)

    type_dct, type_dct_info = get_datatypes_freq(df)
    bool_col_list = findbool(df)
    col_list = (type_dct['float64'])
    col_list_excpt_bool = [column for column in col_list if column not in bool_col_list]
    print('col_list_exc_bool = {}'.format(len(col_list_excpt_bool)))

    df['target'] = y_train
    df['target'] = df['target'].astype(float)
    temp = df.copy()

    t1, t2 = iv_woe(temp[np.append(col_list_excpt_bool,['target'])], 'target', bins=5, show_woe=False)
    feature_list = list(t1[ (t1['IV']<iv_upper_limit) & (t1['IV']>iv_lower_limit) ]['Variable'].values)

    #remove correlated features
    corr_df, subset_df = sel.get_correlated_features(df, feature_list, thresh=corr_arr)
    feature_list = sel.corr_iter(df, np.array(feature_list), thresh=corr_arr)
    feature_list = list(feature_list)

    #vif
    feature_list, vif_df = vif_iter(df, feature_list, threshold=vif_arr)

    #backward feature selection
    feat_list = ft.backward_feature_selection(df[feature_list], y_train, num_features=features_arr)

    return feat_list

In [11]:
feat_df = pd.DataFrame(df_raw.columns,columns = ['feature_name']) 
feat_df['count'] = 0
feat_df

,feature_name,count
0,service_count_6M,0
1,median_payroll_amt_3M,0
2,ratio_third_party_size_1M_6M,0
3,ratio_amt_credits_less_100_credits_grt_500_3M,0
4,ecom_flag,0
...,...,...
207,ratio_median_pos_amt_3M_6M,0
208,payroll_flag,0
209,third_party_count_1M,0
210,third_party_count_3M,0


In [12]:
def increase_counts(dataframe, feature_column, count_column, feat_list):
    dataframe.loc[dataframe[feature_column].isin(feat_list), count_column] += 1
    return dataframe
# increase_counts(feat_df,'feature_name','count',feat_list)
# feat_df.sort_values(by='count',ascending=False)

In [13]:
for seed in range(50):
    feat_list = main_func(df_raw, corr_arr,vif_arr,features_arr,iv_upper_limit,iv_lower_limit,seed)
    feat_df = increase_counts(feat_df,'feature_name','count',feat_list)


x_train.shape = (445, 211) | y_train.shape = (445,) | x_test.shape = (192, 211) | y_test.shape = (192,)
col_list_exc_bool = 191
Information value of service_count_6M is 0.010823
Information value of median_payroll_amt_3M is 0.02717
Information value of ratio_amt_credits_less_100_credits_grt_500_3M is 0.257612
Information value of count_credits_100_to_500_6M is 0.094571
Information value of sum_credits_100_to_500_6M is 0.080183
Information value of count_credits_100_to_500_3M is 0.143433
Information value of ratio_cnt_credits_less_100_credits_grt_500_3M is 0.10952
Information value of shops_count_6M is 0.044836
Information value of median_service_amt_1M is 0.000419
Information value of ratio_pos_size_1M_3M is 0.000498
Information value of ratio_shops_size_credit_size_6M is 0.140271
Information value of median_pos_amt_3M is 0.07475
Information value of ratio_amt_credits_less_100_credits_grt_1500_6M is 0.120326
Information value of sum_credits_lessthan_100_3M is 0.215784
Information value

In [17]:
feat_df

,feature_name,count
0,service_count_6M,0
1,median_payroll_amt_3M,0
2,ratio_third_party_size_1M_6M,0
3,ratio_amt_credits_less_100_credits_grt_500_3M,0
4,ecom_flag,0
...,...,...
207,ratio_median_pos_amt_3M_6M,3
208,payroll_flag,0
209,third_party_count_1M,2
210,third_party_count_3M,4


In [18]:
feature_list = list(feat_df.sort_values(by='count',ascending=False)['feature_name'])

In [21]:
corr_df, subset_df = sel.get_correlated_features(df_raw, feature_list, thresh=corr_arr)
corr_df[np.abs(corr_df['corr'])<0.6]

,feature_1,feature_2,corr
1,ratio_pos_size_credit_size_6M,ratio_pos_size_credit_size_1M,0.591695
3,sum_credits_100_to_500_1M,third_party_count_3M,0.553569
4,sum_credits_100_to_500_1M,sum_credits_lessthan_100_1M,0.556027
7,sum_credits_100_to_500_1M,third_party_count_6M,0.518886
11,sum_credits_100_to_500_1M,sum_credits_lessthan_100_6M,0.537090
...,...,...,...
3304,service_amt_1M,ach_size_1M,0.584952
3305,service_amt_1M,median_ach_amt_6M,0.588152
3307,service_amt_1M,ach_size_6M,0.563547
3309,service_amt_1M,median_ach_amt_1M,0.589072
